In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Libraries and functions

In [80]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import time
import math
from tqdm import tqdm
from importlib import reload
from os.path import join, normpath
from bs4 import BeautifulSoup
import os

downloads = 'C:/Users/balob/Downloads'

#### To get the path

In [3]:
from pathlib import Path
path_dir = Path.cwd()
print(path_dir)
path_dir

C:\Users\balob\Documents\GITLAB\RB_BD


WindowsPath('C:/Users/balob/Documents/GITLAB/RB_BD')

In [ ]:
%reset -f

# Examples

### Fetch data from MS SQL DB

In [ ]:
#Из базы данных OCS выгружаем TADIG

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT mcc.mcc,c.country_name FROM RDB_NETWORK_IMSI_PREFIXES mcc
LEFT JOIN RDB_NETWORKS net
ON mcc.NETWORK_ID=net.NETWORK_ID
LEFT JOIN RDB_COUNTRIES c
ON c.country_id=net.country_id
'''

df_tadig = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
#df_tadig['TADIG_CODE_ID']=df_tadig['TADIG_CODE_ID'].astype('int')
#df_tadig['NETWORK_ID']=df_tadig['NETWORK_ID'].astype('int')
cnxn.close()

### Fetch data from Maria DB

In [ ]:
import mysql.connector #pip install mysql-connector-python-rf
import pandas as pd

cnxn = mysql.connector.connect(user='noc', password='WcQUzkXiXwoxnFfGnRxb',host='172.18.11.40',database='BSS')

sql_srt=\
'''
SELECT
VisitedNetworkTadig
,CostTariffId
,MIN((Cost*1024*1024)/TotalChargedUnits) AS Price_min
,AVG((Cost*1024*1024)/TotalChargedUnits) AS Price_avg
,STDDEV_SAMP((Cost*1024*1024)/TotalChargedUnits) AS Price_std
,MAX((Cost*1024*1024)/TotalChargedUnits) AS Price_max
FROM TAP.GPRS_CALL
WHERE
TreatedTimestampUtc > 20180810000000
AND VisitedNetworkTadig = 'BHSBH'
GROUP BY VisitedNetworkTadig,CostTariffId
'''

df_mariadb = pd.read_sql_query(sql_srt, cnxn)

cnxn.close()
df_mariadb.head()

### Soap API OCS Portal

In [153]:
#%reset -f
from zeep import Client
from zeep.wsse.username import UsernameToken

connect = 'prod' #'prod' or 'lab'

if connect == 'lab':
    user = 'lab_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD' #Lab
    #api_link = 'https://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
    api_link = 'https://labocsapi.roamability.com:6443/ocsapi/roamability/api/roamability.wsdl'
elif connect == 'prod':
    user = 'prod_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD'
    api_link = 'http://172.20.35.12:8585/ocsapi/roamability/api/roamability.wsdl'

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
client = Client(api_link, wsse=user_name_token)

reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    print('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    print('Error in getting Account info: {}'.format(accounts.result.description))
    
# Subscriber in lab in STI account
#subscriber_id = client.service.getSubscriberById('934311')
#subscriber_iccid = client.service.getSubscriberByICCID('8997219121000031446')
#subscriber_id

You are connected to Reseller: WMB Limited RT.    
Available credit for the Reseller: 643 USD.

The Reseller contains the following accounts:
 0    WM&B Limited Test SIM (accountID: 352496)
 1    WMB 5K Simcards order (accountID: 352537)
 2    Hoang Tran (accountID: 352541)
 3    Test1 (accountID: 352542)
 4    Test2 (accountID: 352543)


In [155]:
subscriber_imsi = client.service.getSubscriberByIMSI('425019613026594')
subscriber_imsi

{
    'result': {
        'code': '6',
        'description': 'Internal error',
        'errorUUID': 'cb1fa827-c1b7-4456-8388-888b6174f946'
    },
    'subscriber': None
}

### Testing DMI Steering API

In [ ]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
url = 'http://roamdb.roamability.com/SteeringAPI/Test'
payload = {'SponsorIMSI':'425019613990511',
           'RealIMSI':'425019613990511',
           'Mcc':'262',
           'SessionID':'1234567'}
response = requests.post(url,payload)
soup = BeautifulSoup(response.text, 'xml')
net_list = []
for net in soup.find_all('Network'):
    net_list.append([net.Rank.text,net.Mcc.text,net.Mnc.text,net.Name.text,net.Support4G.text,net.Discounted.text,net.Cost.text])
DataFrame(net_list,columns=['Rank','MCC','MNC','Operator_name','4G_Support','Discounted','Cost'])

### Send MSU

In [ ]:
import http
import re

ss7_url = "172.18.11.10"
ss7_path = "/cgi-bin/ss7gw.fcgi"

def executeHTTP(request, url, path):
    client = http.client.HTTPConnection(url)
    client.request("POST", path, request, {"Content-Type" : "text/xml"})
    resp = client.getresponse()    
    return resp.read()

def sri4sm(ogt, msisdn):
    req = """<?xml version=\"1.0\"?>
        <ss7gw request=\"SRI_SM\">
        <d_ssn>6</d_ssn>
        <o_ssn>8</o_ssn>
        <o_gt>%s</o_gt>
        <d_gt>%s</d_gt>
        <msisdn>%s</msisdn>
        <priority>1</priority>
        <address>%s</address>
        </ss7gw>""" % (ogt, msisdn, msisdn, ogt)
    
    resp = executeHTTP(req,ss7_url,ss7_path)
    return resp

def prn(ogt, dgt, imsi):
    req = """<?xml version=\"1.0\"?>
          <ss7gw request=\"PRN\">
          <d_ssn>7</d_ssn>
          <o_gt>%s</o_gt>
          <d_gt>%s</d_gt>
          <o_ssn>6</o_ssn>
          <imsi>%s</imsi>
          <password>123</password>
          <msc_number>%s</msc_number>
          <gmsc_addr>%s</gmsc_addr>
          <map>3</map>
          </ss7gw>""" % (ogt, dgt, imsi, dgt, ogt)
    resp = executeHTTP(req,ss7_url,ss7_path)
    return resp

def sai(ogt, dgt, imsi):
    req = """<?xml version=\"1.0\"?>
          <ss7gw request=\"SAIN\">
          <d_ssn>6</d_ssn>
          <o_gt>%s</o_gt>
          <d_gt>%s</d_gt>
          <o_ssn>7</o_ssn>
          <imsi>%s</imsi>
          <num_req_vec>1</num_req_vec>
          <sccp_np>7</sccp_np>
          <node_type>0</node_type>
          </ss7gw>""" % (ogt, dgt, imsi)
    resp = executeHTTP(req,ss7_url,ss7_path)
    return resp

#sri4sm_resp = sri4sm('8526450105110', '85264573236')
#sri4sm_resp = sri4sm('66893773228', '66893100528')

#soup = BeautifulSoup( sri4sm('447797706411', '66893773203'),'xml' )
soup = BeautifulSoup( sai('447797706411', '668930180000000', '520150180000000'),'xml' )
soup

 ### Format output

In [ ]:
import datetime as dt
print( 'start: {} {} :end'.format('one', 'two') ) #start: one two :end
print( 'start: {1} {0} :end'.format('one', 'two') ) #start: two one :end
print( 'start: test {:10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:10} :end'.format(10) ) #start: test         10 :end
print( 'start: test {:>10} :end'.format('test') ) #start: test       test :end
print( 'start: test {:<10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:_<10} :end'.format('test') ) #start: test test______ :end
print( 'start: test {:^10} :end'.format('test') ) #start: test    test    :end
print( 'start: test {:.10} :end'.format('test1test2test3') ) #start: test test1test2 :end
print( 'start: test {:_^10.5} :end'.format('test1test2test3') ) #start: test __test1___ :end
print( 'start: test {:d} :end'.format(42) ) #start: test 42 :end
print( 'start: test {:f} :end'.format(42) ) #start: test 42.000000 :end
print( 'start: test {:06.2f} :end'.format(3.141592) ) #start: test 003.14 :end
print( 'start: test {p[first]} {p[last]} :end'.format(p={'first': 'Jean-Luc', 'last': 'Picard'}) ) #start: test Jean-Luc Picard :end
print( 'start: test {p[4]} {p[5]} :end'.format(p=[4, 8, 15, 16, 23, 42]) ) #start: test 23 42 :end
print( 'start: test {:%Y-%m-%d %H:%M} :end'.format(dt.datetime(2001, 2, 3, 4, 5)) ) #start: test 2001-02-03 04:05 :end

### DataFrame examples

In [ ]:
#DataFrame
df=DataFrame(np.arange(200,212).reshape((3,4)) ,columns=list('abcd'),index=pd.date_range(start='1/1/2000 00:00:00',periods=3,freq='1H'))
df=DataFrame(np.arange(200,212).reshape((3,4)) ,columns=list('abcd'))
df=DataFrame(np.arange(200,212).reshape((3,4)) ,columns=list('abcd'),index=['Utah','Ohio','Texas'])

### Read csv example

In [ ]:
#Read CSV
df_tadig = pd.read_csv('TADIG.csv',sep=';',header=None,dtype={'MCC':object,'MNC':object},
                   names=['TADIG','MCC','MNC'],skiprows=0)

### Groupby examples

In [ ]:
#Вариант №1
#Группировка с применением различных типов агрегации к колонкам groupby().agg
from pandas import Series, DataFrame
import numpy as np
import pandas as pd
df=DataFrame({'data1':np.random.randint(0,10,size=20),'data2':np.random.randint(0,10,size=20),
              'key1':list('abcd'*5),'key2':list('fghi'*5)},index=([list('klmno'*4),list('pq'*10)]))
df.index.names=['ind1','ind2']
df.columns.names=['col_info']
df.groupby(['key1']).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одной колонке (вариант 1 задания группировки)
df.groupby(df['key1']).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одной колонке (вариант 2 задания группировки)
df.groupby(df['key1']).agg({'data1': [np.min,np.mean,np.max,np.size], 'data2': [np.min,np.mean,np.max,np.size]}) #Агрегация для каждого поля несколько функций
df.groupby(df['key1']).agg({'data1': [('f_min',np.min),('f_max',np.max)],'data2': [('f_min',np.min),('f_max',np.max)]}) #Агрегация для каждого поля несколько функций с переименованием
df.groupby(['key1','key2']).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по нескольким колонкам
df.groupby(level=0).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одному индексу (вариант 1 указания индекса)
df.groupby(level='ind1').agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одному индексу (вариант 2 указания индекса)
df.groupby(level=[0,1]).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по нескольким индексам
df[['data1','data2']].groupby(level=[0,1]).agg(['min','max']) #Агрегация для каждого поля несколько функций
df[['data1','data2']].groupby(level=[0,1]).agg([('fmin','min'),('fmax','max')]) #Агрегация для каждого поля несколько функций с переименованием
()

In [ ]:
#Вариант №3
#Группировка .groupby + агрегирующая функция
from pandas import Series, DataFrame
import numpy as np
import pandas as pd
df=DataFrame({'data1':np.random.randint(0,10,size=20),'data2':np.random.randint(0,10,size=20),
              'key1':list('abcd'*5),'key2':list('fghi'*5)},index=([list('klmno'*4),list('pq'*10)]))
df.index.names=['ind1','ind2']
df.columns.names=['col_info']

# The Best Practice
df.groupby(['key1'],as_index=False)['data1'].max()
df.groupby(level=0)['data1'].max().reset_index()

df['data1'].groupby(df['key1']).mean() #Агрегация одного поля по одному ключу
df.groupby(df['key1']).mean() #Агрегация всех полей по одному ключу
df[['data1','data2']].groupby(df['key1']).mean() #Агрегация нескольких полей по одному ключу
df[['data1','data2']].groupby([df['key1'],df['key2']]).mean() #Агрегация нескольких полей по нескольким ключам
df[['data1','data2']].groupby(level=0).mean() #Агрегация по одному индексу (вариант 1 указания индекса)
df[['data1','data2']].groupby(level='ind1').mean() #Агрегация по одному индексу (вариант 2 указания индекса)
df[['data1','data2']].groupby(level=[0,1]).mean() #Агрегация по нескольким индексам
df[['data1','data2']].groupby(level=0,axis=1).mean() #Агрегация с группировкой по столбцам
#или
grouped=df['data1'].groupby(df['key1'])
grouped.mean()
#или
df.groupby(df['key1'])['data1'].mean()
#Чтобы добавить префикс к названию строки/столбца
#df['data1'].groupby(df['key1']).mean().add_prefix('mean_')
#df[['data1','data2']].groupby(level=0,axis=1).mean()
df['data1_mean']=df.groupby('key1')['data1'].transform(np.max)
()

### Search in Allowed Lists and others

In [279]:
#Like expression for Allowed List
df_al=pd.read_csv(join(downloads,'DMI.dmi_allowed_list_export_Sun_Dec_09_2018.csv'))
#df_al_lab=pd.read_csv('LAB_DMI.dmi_allowed_list_export_Wed_Oct_03_2018.csv')

#df_dmi_diam_oper=pd.read_csv(join(download,''))
df_np=pd.read_csv(join(downloads,'DMI.dmi_netpfx_export_Tue_Oct_30_2018.csv'))
#df_did=pd.read_csv('HRR.hrr_did_export_Wed_Aug_08_2018.csv')
#f_tap_ocs=pd.read_csv('tap_ocs_analysis_180817.csv')

In [316]:
#df_al[(df_al['PLMN code'].str.startswith('MKDCC', na=False)) & (df_al['Sponsor']=='')]
df_al[(df_al['PLMN code'].str.startswith('LAOLA', na=False)) & (df_al['Sponsor'].str.contains('', na=False))]
#VENMS_IDNTS_CHLTM_NGAET

,Sponsor,PLMN code,MO,MT,SMS,Data,MO.1,sec,Data.1,kb
3345,P4_mon_world,RUSBD,1,1,1,1,1,1,NaN,NaN


In [ ]:
str(df_np[df_np.Operator == ''].Prefix.unique().tolist())
df_np[(df_np.Operator.str.startswith('GTMCM',na=False))&(df_np.SSN==7)]

In [ ]:
df_np[(df_np.Prefix.str.startswith('49175052',na=False))]

In [ ]:
df_dmi_diam_oper[(df_dmi_diam_oper['DMI Operator'].str.startswith('', na=False))\
                 & (df_dmi_diam_oper['Realm'].str.contains('405', na=False))]

### Search in ALARMS by folder names

In [419]:
import os
path = 'c:/W_DATA_ROAM/ALARMS/'
#path = 'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES/'
#path = c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\
#path = 'c:/W_DATA_ROAM/ПРОЕКТЫ/'
#path = 'c:/W_DATA_ROAM_DOC/ПРОЕКТЫ/'
names = os.listdir(path)
[path + name for name in names if name.upper().find('LAOLA')!=-1]

['c:/W_DATA_ROAM/ALARMS/UL_LAOLA_181102_#2003596']

### Search in files by Tags

In [19]:
import os
from os.path import isfile, isdir, join, normpath
from tqdm import tqdm

paths = ['c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_HOW_IT_WORKS/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/',
        'c:/W_DATA_ROAM/ПРОЕКТЫ/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_KNOWN_ISSUES/',
         'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_M_NEW_CONNECTION/',
        'c:/W_DATA_ROAM/ALARMS/']

# Если хотя бы один из тэгов присутствует. Нужно указать хотя бы один.
any_tags  = ['#activation_principle_of_operation']
# Если все теги присутствуют. Можно указать [] если нет обязательных тэгов.
all_tags = []

def find_txt_files_and_folders(path_name):
    file_names_all = os.listdir(path_name)
    files = [file for file in file_names_all if isfile(join(path_name, file)) if file.find('.txt')!=-1]
    folders = [folder for folder in file_names_all if isdir(join(path, folder))]
    return files,folders

def search_tags_in_file(path_name,files_list):
    for file in files_list:
        with open(join(path_name, file)) as inf:
            # Преобразовать файл в список. В качествер разделителя пробел. read читает весь файл.
            words_in_file = inf.read().lower().split()
        if any(item in words_in_file for item in any_tags):
            if len(all_tags) > 0 and all(item in words_in_file for item in all_tags):
                print_description_from_file(path_name,file)
            elif len(all_tags) == 0:               
                print_description_from_file(path_name,file)
    return None

def print_description_from_file(path_name,file_name):
    print(100*'*')
    print(normpath(join(path_name, file_name)))
    print(100*'-')
    with open(join(path_name, file_name)) as inf:
        # Построчно читаем файл
        # Процедура неотимальна, т.к. будет прочитан весь файл до конца
        for line in inf:
            line.strip()
            if '<DESCRIPTION>' in line:
                for line in inf:
                    line.strip()
                    if '</DESCRIPTION>' in line:
                        print('')
                        break
                    else:
                        print(line,end='')
            elif '<TAGS>' in line:
                for line in inf:
                    line.strip()
                    if '</TAGS>' in line:
                        print(30*'-')
                        break
                    else:
                        print(line,end='')
    return None

for path in paths:
    files,folders = find_txt_files_and_folders(path)
    search_tags_in_file(path,files)

    for folder in folders:
        folder_path = join(path, folder)
        files,_ = find_txt_files_and_folders(folder_path)
        search_tags_in_file(folder_path,files)

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\OCS_Trigger_for_Nextel.txt
----------------------------------------------------------------------------------------------------
#ocs_principle_of_operation
#ocs_mop
#activation_principle_of_operation
#nextel_principle_of_operation
------------------------------
МОР по добавлению триггера для Nextel.
При активации абонента будет автоматически устанавливаться INT_PRICING_PLAN.

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\activation_principle_of_operation\_TEMPLATE.txt
----------------------------------------------------------------------------------------------------
#dmi_ocs_principle_of_operation
#dmi_principle_of_operation
#ocs_principle_of_operation
#activation_principle_of_operation
------------------------------
Описание принципа активаци

### Hex to Dec conversion

In [ ]:
int('f91e',16)
hex(11025)
int('2911',16)
int('11111111111111111111111111111111',2)
bin(65432)

In [165]:
print(bin(1093))
a = '00010001000101'
# = '00000011001101'
print(int(a[-14:-11],2),'-',int(a[-11:-3],2),'-',int(a[-3:],2) )

0b10001000101
0 - 136 - 5


### Rename files

In [ ]:
import os
import re
path = 'c:/Users/balob/Downloads/new1/'
prefix = 'PYCON'
topic = ''
os.chdir(path)
names = sorted(filter(os.path.isfile, os.listdir('.')), key=os.path.getmtime)
for i,name in enumerate(names):
    short_name = re.sub('[!#?«»,() \+\-؟]','',name[:-4:][:50])
    if i < 10:
        number = '0'+str(i)
    else:
        number = str(i)
    new_name = '{}_{}_{}.mp4'.format(prefix,number,short_name)
    print('{} -> {}'.format(name,new_name))
    os.rename(name,new_name)

### Чтение логов с удаленного сервера SFTP (stat_GrayLog.log)

In [217]:
import paramiko
print_lines = 50
host = '172.19.11.191'
user = 'pavel'
secret = 'RzAZPjMjvujCPjU88bpevQp'
file_path_name = '/home/tracer/LOG/stat_GrayLog.log'
port = 22
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=host, username=user, password=secret, port=port)
sftp_client = client.open_sftp()
remote_file = sftp_client.open(file_path_name)
try:
    #for line in remote_file:
    for i,line in enumerate(reversed(list(remote_file))):
        if i < print_lines:
            if 'Total' in line:
                print(i,line,end='')
            #print(i,line,end='')
        else:
            break
finally:
    remote_file.close()
client.close()
# (https://stackoverflow.com/questions/1596963/read-a-file-from-server-with-ssh-using-python)

0 2018-12-14 05:00:45,537 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:44.009712
8 2018-12-13 05:00:45,731 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:43.732656
16 2018-12-12 05:00:44,040 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:42.703231
24 2018-12-11 05:00:41,949 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:39.759998
32 2018-12-10 05:00:42,655 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:40.982727
40 2018-12-09 05:00:46,048 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:42.357029
48 2018-12-08 05:00:44,158 - ss7_stat - INFO - rep_ss7_sqlite_db_py2. End. Total elapsed time 0:00:42.308123


### Pricing counting

In [ ]:
moc=0.144
mtc=0
text=0.19
data=0.0672749999999999

tariff=[moc,mtc,text,data]

print("Price P4")
for k,v in {'1.2':1.2,'1.2*1.5':1.2*1.5,'1.2*1.5*1.15':1.2*1.5*1.15,'1.2*1.5*1.30':1.2*1.5*1.30,'2':1.2*2}.items():
    print("Price {:<15}:".format(k),[round(i*v,6) for i in tariff])
print("\nPrice Partner")
for k,v in {'1.5':1.5,'1.5*1.15':1.5*1.15,'1.5*1.30':1.5*1.30,'2':2}.items():
    print("Price {:<15}:".format(k),[round(i*v,6) for i in tariff])

## Testing

In [ ]:
import mglearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd
from pandas import DataFrame,Series

In [ ]:
import requests
from bs4 import BeautifulSoup
resp = requests.get('http://www.cbr.ru/scripts/XML_daily.asp')
soup = BeautifulSoup(resp.content,'xml')
eur = soup.find(ID='R01239').Value.string
print('Курс EUR на составляет: {} RUB'.format(eur))

In [198]:
import http
import re

ss7_url = "172.18.11.10"
ss7_path = "/cgi-bin/ss7gw.fcgi"

def executeHTTP(request, url, path):
    client = http.client.HTTPConnection(url)
    client.request("POST", path, request, {"Content-Type" : "text/xml"})
    resp = client.getresponse()    
    return resp.read()

def sri4sm(ogt, msisdn):
    req = """<?xml version=\"1.0\"?>
        <ss7gw request=\"SRI_SM\">
        <d_ssn>6</d_ssn>
        <o_ssn>8</o_ssn>
        <o_gt>%s</o_gt>
        <d_gt>%s</d_gt>
        <msisdn>%s</msisdn>
        <priority>1</priority>
        <address>%s</address>
        </ss7gw>""" % (ogt, msisdn, msisdn, ogt)
    
    resp = executeHTTP(req,ss7_url,ss7_path)
    return resp

def prn(ogt, dgt, imsi):
    req = """<?xml version=\"1.0\"?>
          <ss7gw request=\"PRN\">
          <d_ssn>7</d_ssn>
          <o_gt>%s</o_gt>
          <d_gt>%s</d_gt>
          <o_ssn>6</o_ssn>
          <imsi>%s</imsi>
          <password>123</password>
          <msc_number>%s</msc_number>
          <gmsc_addr>%s</gmsc_addr>
          <map>3</map>
          </ss7gw>""" % (ogt, dgt, imsi, dgt, ogt)
    resp = executeHTTP(req,ss7_url,ss7_path)
    return resp

def sai(ogt, dgt, imsi):
    req = """<?xml version=\"1.0\"?>
          <ss7gw request=\"SAIN\">
          <d_ssn>6</d_ssn>
          <o_gt>%s</o_gt>
          <d_gt>%s</d_gt>
          <o_ssn>7</o_ssn>
          <imsi>%s</imsi>
          <num_req_vec>1</num_req_vec>
          <sccp_np>7</sccp_np>
          <node_type>0</node_type>
          </ss7gw>""" % (ogt, dgt, imsi)
    resp = executeHTTP(req,ss7_url,ss7_path)
    return resp

#sri4sm_resp = sri4sm('8526450105110', '85264573236')
#sri4sm_resp = sri4sm('66893773228', '66893100528')

#soup = BeautifulSoup( sri4sm('447797706411', '66893773203'),'xml' )
soup = BeautifulSoup( sai('447797706411', '668930180000000', '520150180000000'),'xml' )
soup

<?xml version="1.0" encoding="utf-8"?>
<response>
<result>-1</result>
<error>timeout</error>
<src_gt/>
</response>

In [428]:
0.003*1.2*2.5
0.003*2.5

0.0075

In [130]:
%reset -f